In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
df = pd.read_csv("../data/raw/bank-additional-full.csv", delimiter=";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df.shape

(41188, 21)

Rename columns with dots in column name

In [4]:
df = df.rename(columns={'emp.var.rate': 'emp_var_rate'})

In [5]:
df = df.rename(columns={'cons.price.idx': 'cons_price_idx'})

In [6]:
df = df.rename(columns={'cons.conf.idx': 'cons_conf_idx'})

In [7]:
df = df.rename(columns={'nr.employed': 'nr_employed'})

Drop duration because we cannot know the duration before a call is made.

In [8]:
df = df.drop("duration", axis=1)
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [9]:
df.nr_employed.unique()

array([5191. , 5228.1, 5195.8, 5176.3, 5099.1, 5076.2, 5017.5, 5023.5,
       5008.7, 4991.6, 4963.6])

In [10]:
df.emp_var_rate.unique()

array([ 1.1,  1.4, -0.1, -0.2, -1.8, -2.9, -3.4, -3. , -1.7, -1.1])

In [11]:
df = df.drop_duplicates()
df.shape

(39404, 20)

In [12]:
df = df.dropna()

Setting impute strategy

In [13]:
impute_missing = SimpleImputer(missing_values="unknown", strategy="most_frequent")

Check values for each column. If more than 10% values are "unknown" we will impute the row containing that value, otherwise rows will e deleted.

In [14]:
df.job.value_counts()

admin.           9873
blue-collar      8835
technician       6404
services         3801
management       2820
retired          1683
entrepreneur     1405
self-employed    1386
housemaid        1028
unemployed        992
student           852
unknown           325
Name: job, dtype: int64

In [15]:
df = df.drop(df[df.job == "unknown"].index)
df.shape

(39079, 20)

In [16]:
df.marital.value_counts()

married     23639
single      10923
divorced     4447
unknown        70
Name: marital, dtype: int64

In [17]:
df = df.drop(df[df.marital == "unknown"].index)
df.shape

(39009, 20)

In [18]:
df.default.value_counts()

no         30898
unknown     8108
yes            3
Name: default, dtype: int64

In [19]:
df["default"] = impute_missing.fit_transform(df[["default"]])
df.default.value_counts()

no     39006
yes        3
Name: default, dtype: int64

In [20]:
df.loan.value_counts()

no         31923
yes         6112
unknown      974
Name: loan, dtype: int64

In [21]:
df = df.drop(df[df.loan == "unknown"].index)
df.shape

(38035, 20)

In [22]:
df.housing.value_counts()

yes    20365
no     17670
Name: housing, dtype: int64

In [23]:
df = df.drop(df[df.housing == "unknown"].index)
df.shape

(38035, 20)

In [24]:
df.age.value_counts()

31    1774
32    1699
33    1674
36    1624
35    1613
      ... 
89       2
91       2
87       1
94       1
95       1
Name: age, Length: 78, dtype: int64

In [25]:
df.education.value_counts()

university.degree      11219
high.school             8853
basic.9y                5599
professional.course     4875
basic.4y                3820
basic.6y                2134
unknown                 1517
illiterate                18
Name: education, dtype: int64

In [26]:
df = df.drop(df[df.education == "unknown"].index)
df.shape

(36518, 20)

In [27]:
df.contact.value_counts()

cellular     23012
telephone    13506
Name: contact, dtype: int64

In [28]:
df.month.value_counts()

may    12356
jul     6177
aug     5372
jun     4775
nov     3709
apr     2342
oct      649
sep      496
mar      482
dec      160
Name: month, dtype: int64

In [29]:
df.day_of_week.value_counts()

mon    7611
thu    7603
wed    7237
tue    7110
fri    6957
Name: day_of_week, dtype: int64

In [30]:
df.campaign.value_counts()

1     15143
2      9483
3      4848
4      2437
5      1489
6       906
7       578
8       372
9       258
10      206
11      166
12      115
13       75
14       64
17       54
15       47
16       45
18       31
20       30
19       23
21       19
22       17
23       16
24       15
27       11
29        9
28        8
31        7
26        7
25        7
30        7
35        5
33        4
32        4
34        3
40        2
42        2
43        2
37        1
39        1
41        1
Name: campaign, dtype: int64

In [31]:
df.pdays.value_counts()

999    35153
3        392
6        379
4        106
2         57
9         55
12        53
7         52
5         45
10        44
13        33
11        26
1         24
15        22
14        18
0         15
8         14
16         8
17         6
18         5
19         3
22         3
21         2
26         1
25         1
27         1
Name: pdays, dtype: int64

In [32]:
df.previous.value_counts()

0    31413
1     4161
2      671
3      195
4       57
5       16
6        4
7        1
Name: previous, dtype: int64

In [33]:
df.poutcome.value_counts()

nonexistent    31413
failure         3863
success         1242
Name: poutcome, dtype: int64

In [34]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx',
       'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [35]:
df.emp_var_rate.value_counts()

 1.4    14158
-1.8     8300
 1.1     6880
-0.1     3331
-2.9     1467
-3.4      953
-1.7      699
-1.1      570
-3.0      150
-0.2       10
Name: emp_var_rate, dtype: int64

In [36]:
df.cons_price_idx.value_counts()

93.994    6880
93.918    5738
92.893    5294
93.444    4488
94.465    3932
93.200    3268
93.075    2193
92.201     682
92.963     627
92.431     399
92.649     325
94.215     281
94.199     267
92.843     254
92.379     229
93.369     228
94.055     216
94.027     202
94.601     187
93.876     182
92.469     158
92.713     150
93.749     149
94.767     116
93.798      63
92.756      10
Name: cons_price_idx, dtype: int64

In [37]:
df.cons_conf_idx.value_counts()

-36.4    6880
-42.7    5738
-46.2    5294
-36.1    4488
-41.8    3932
-42.0    3268
-47.1    2193
-31.4     682
-40.8     627
-26.9     399
-30.1     325
-40.3     281
-37.5     267
-50.0     254
-29.8     229
-34.8     228
-39.8     216
-38.3     202
-49.5     187
-40.0     182
-33.6     158
-33.0     150
-34.6     149
-50.8     116
-40.4      63
-45.9      10
Name: cons_conf_idx, dtype: int64

In [38]:
df.euribor3m.value_counts()

4.857    2542
4.962    2220
4.963    2144
4.961    1679
1.405    1032
         ... 
3.053       1
1.045       1
0.937       1
3.669       1
3.329       1
Name: euribor3m, Length: 314, dtype: int64

In [39]:
df.nr_employed.value_counts()

5228.1    14158
5099.1     7741
5191.0     6880
5195.8     3331
5076.2     1467
5017.5      953
4991.6      699
4963.6      570
5008.7      559
5023.5      150
5176.3       10
Name: nr_employed, dtype: int64

In [40]:
df.y.value_counts()

no     32300
yes     4218
Name: y, dtype: int64

In [41]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


## Final clean data set

In [42]:
df.to_csv("../data/clean/bank-additional-clean.csv", index=False)
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
